In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shayanfazeli/heartbeat --unzip

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.engine.input_layer import Input
from tensorflow.python.ops.gen_array_ops import shape
from keras.utils.np_utils import to_categorical
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, concatenate
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Dane Treningowe
train_raw = pd.read_csv("mitbih_train.csv", header=None)
print(train_raw.head())
print(train_raw.describe())

print("Rozklad klas w danych treningowych")
train_raw[187] = train_raw[187].astype(int)
train_class_spread = train_raw.pivot_table(index = [187], aggfunc ='size')
print(train_class_spread)
sns.catplot(x = 187, kind = 'count', data = train_raw)

In [ ]:
# Dane Testowe
test_raw = pd.read_csv("mitbih_test.csv", header=None)
print(test_raw.head())
print(test_raw.describe())

print("Rozklad klas w danych testowych")
test_raw[187] = test_raw[187].astype(int)
test_class_spread = test_raw.pivot_table(index = [187], aggfunc ='size')
print(test_class_spread)
sns.catplot(x = 187, kind = 'count', data = test_raw)


In [ ]:
# Ze wzgledu na rozklad klas w danych treningowych, nalezy zbalansowac ilosc klas:
# 
# N: Non Ectopic beats (Normal beats)
# S: Supraventrical ectopic beats
# V: Ventricular ectopic beats
# F: Fusion beats
# Q: Unknown beats
#
# Klasa: 0 = 72471
# Klasa: 1 =  2223
# Klasa: 2 =  5788
# Klasa: 3 =   641
# Klasa: 4 =  6431
#
# - metoda upsamplingu i downsamplingu - ujednolicenie danych na poziomie ...

# Poki uzyte zostana wszystkie dane!!! 

In [ ]:
Y_train = to_categorical(train_raw[187].values)
X_train = np.array(train_raw.iloc[:,:-1].values)
# X = np.array(train_raw.iloc[:,-1:].values)[..., np.newaxis]

Y_test = to_categorical(test_raw[187].values)
X_test = np.array(test_raw.iloc[:,:-1].values)
# X_test = np.array(train_raw.iloc[:,-1:].values)[..., np.newaxis]

In [ ]:
print(X_train)
print(Y_train)

print(X_train.shape)
print(Y_train.shape)

print(X_test.shape)
print(Y_test.shape)

In [ ]:
X_train = X_train.reshape(len(X_train),X_train.shape[1],1)
X_test = X_test.reshape(len(X_test),X_test.shape[1],1)

In [ ]:
print(X_train.shape)
print(Y_train.shape)

print(X_test.shape)
print(Y_test.shape)

In [ ]:
def conv_unit(unit, input_layer):
    s = '_' + str(unit)
    layer = keras.layers.Conv1D(name='Conv1' + s, filters=32, kernel_size=5, strides=1, padding='same', activation='relu')(input_layer)
    layer = keras.layers.Conv1D(name='Conv2' + s, filters=32, kernel_size=5, strides=1, padding='same', activation=None)(layer )
    layer = keras.layers.Add(name='ResidualSum' + s)([layer, input_layer])
    layer = keras.layers.Activation("relu", name='Act_relu' + s)(layer)
    layer = keras.layers.MaxPooling1D(name='MaxPool' + s, pool_size=5, strides=2)(layer)
    return layer

def get_uncompiled_model():

    inputs = keras.layers.Input(shape=(187,1), name="Inputo")
    current_layer = keras.layers.Conv1D(name='Conv1D_1', filters=32, kernel_size=5, strides=1, padding='same', input_shape=(187,1))(inputs)

    for i in range(5):
        current_layer = conv_unit(i + 1, current_layer)

    current_layer = keras.layers.Flatten()(current_layer)
    
    current_layer = keras.layers.Dense(32, name='FC1', activation='relu')(current_layer)
    current_layer = keras.layers.Dense(32, name='FC2')(current_layer)
    output_layer = keras.layers.Dense(5, name='Output', activation='softmax')(current_layer)

    model = keras.Model(inputs=inputs, outputs=output_layer)

    return model

def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        optimizer='adam', 
        loss=tf.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    return model

In [ ]:
model = get_compiled_model()
file_path = "baseline_cnn_mitbih.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_accuracy", mode="max", patience=10, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_accuracy", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

In [ ]:
for layer in model.layers:
    print(str(layer.name) + " : " + str(layer.input_shape) + " >> X >> " + str(layer.output_shape))
    # print(layer.output_shape)

In [ ]:
history = model.fit(X_train, Y_train, epochs = 10, callbacks=callbacks_list, validation_data=(X_test, Y_test))

In [ ]:
pd.DataFrame(history.history)
pd.DataFrame(history.history)[['accuracy', 'val_accuracy']].plot()
pd.DataFrame(history.history)[['loss', 'val_loss']].plot()

In [ ]:
# Predicted o/p will be in probability distribution 
predict = model.predict(X_test)

# distributional probability to integers
yhat = np.argmax(predict, axis = 1)

confusion_matrix(np.argmax(Y_test, axis = 1), yhat)

plt.figure(figsize=(8,6))
sns.heatmap(confusion_matrix(np.argmax(Y_test, axis =1), yhat), annot = True, fmt = '0.0f', cmap= 'RdPu')

print(classification_report(np.argmax(Y_test, axis=1), yhat))